## P4-1 NET-PL mit CRAG_v2

In [ ]:
!pip install git+https://github.com/rtqichen/torchdiffeq

## Netz auswählen

In [ ]:
TRAIN_RESNET = False
TRAIN_UNODE = False
TRAIN_UNET = True

def get_title():
    if TRAIN_UNODE: return 'U-NODE'
    elif TRAIN_RESNET: return 'RESNET'
    elif TRAIN_UNET: return 'UNET'

## Biliotheken

---

In [ ]:
import os
import glob
import random

import torch
import torch.utils.data

import PIL
import skimage.measure
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm, tqdm_notebook

%matplotlib inline

from models import ConvODEUNet, ConvResUNet, ODEBlock, Unet
#from dataloader import GLaSDataLoader
from dataloader_crag import Crag_DataLoader
from train_utils import plot_losses

from IPython.display import clear_output



##Konstanten

In [ ]:
SHAPE_IMG=(3,512, 512) # h=512 # 1024 #752 #758  #752 # orginal 1516 # w=512 # 1024 #752 #754  #752  # orginal 1509
SHAPE_MASK=(512, 512)

EP=200 #600               # Epochen
LR=0.001             # Lernrate  0.0001 lernrate original
BL=5                 # Bloecke

## Download the filnames of dataset

MILD-Net: "Colorectal Adenocarcinoma Gland (CRAG) Dataset"

https://warwick.ac.uk/services/its/intranet/projects/webdev/sandbox/juliemoreton/research-copy/tia/data/mildnet

Datenset zum Herunterladen:

https://drive.google.com/u/0/uc?id=1p3dZXpgeA1IcGO6vXhStbVLMku-fZTmQ&export=download

In [ ]:
if not os.path.exists('CRAG_v2'):
    print('Bitte laden sie das Datenset in das Projektverzeichnis!')
    print('Das Verzeichnis lautet "CRAG_v2".')

In [ ]:
from jh_path import Path as PATH   # Pfade und Dateinamen
path=PATH() # Instanz der Klasse für Methodenaufruf erforderlich

path_images=PATH.dataset / 'train/Images/'
path_targets=PATH.dataset / 'train/Annotation/'
# input and target files
filenames_inputs  =path.get_filenames(path=path_images , dateifilter= '*.png')
filenames_targets =path.get_filenames(path=path_targets ,dateifilter='*.png')
# sicherstellen, dass beide Listen die gleiche Länge besitzen.
print('Anzahl der Bilder      : ', len(filenames_inputs))
print('Anzahl der Annotationen: ', len(filenames_targets))

## Dateipfade aufteilen in Trainings- und Validierungsdateipfade

In [ ]:

torch.manual_seed(0)

val_set_idx = torch.LongTensor(10).random_(0, len(filenames_inputs)) #+
train_set_idx = torch.arange(0, len(filenames_inputs)) #+

overlapping = (train_set_idx[..., None] == val_set_idx).any(-1)
train_set_idx = torch.masked_select(train_set_idx, ~overlapping)


## Define datasets and dataloaders #+
- Datenset WARWIEK qu: Originalbilder (hxw) 522 x 775 (1,485)-> scaliert auf 352x512 (1,455), scal:1,48/1.58
- Datenset CRAG_v2   : Originalbilder (hxw) 1516x1509 (0,995)-> skaliert auf a), b), c)
- a) ->  352x512 (1,455), scal: 
- c) ->  752x752 (1,000), scal: 2,0
- d) -> 1024x1024 (1)  , scal: 1.5
- e) ->  512 x512 (1) , scal: 2,95
- f) ->  256 x256 (1)  , scal: 5,9

num_workers hardwareabhängig auf 8 gesetzt

In [ ]:


trainset = Crag_DataLoader(filenames_inputs,filenames_targets,SHAPE_MASK, dataset_repeat=1, images=train_set_idx)
valset = Crag_DataLoader(filenames_inputs,filenames_targets,SHAPE_MASK, dataset_repeat=1, images=val_set_idx, validation=True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=4)   ## Batchsize größer setzen? Batchsize original 1
valloader = torch.utils.data.DataLoader(valset, batch_size=4, shuffle=False, num_workers=4)


# Plotting train data

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=6, figsize=(24, 15))

for y in range(5): # y ist idx einer Stichprobe
    for x in range(3):  
        sample = trainset[y]
        ax[y, x * 2].imshow(sample[0].numpy().transpose(1,2,0))
        ax[y, x * 2 + 1].imshow(sample[1][0])
        ax[y, x * 2].axis('off')
        ax[y, x * 2 + 1].axis('off')

plt.show();

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(24, 15))

sample = trainset[0]
ax[1].imshow(sample[1][0].numpy())
ax[2].imshow(sample[1].sum(dim=0))
ax[0].imshow(sample[0].numpy().transpose(1,2,0))

# Plotting validation data

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=6, figsize=(24, 15))

for y in range(5):
    for x in range(3):
        sample = valset[y]
        ax[y, x * 2].imshow(sample[0].numpy().transpose(1,2,0))
        ax[y, x * 2 + 1].imshow(sample[1][1])
        ax[y, x * 2].axis('off')
        ax[y, x * 2 + 1].axis('off')

plt.show(); 

# Define network

In [ ]:
devicename='cuda'
device = torch.device(devicename)

if TRAIN_UNODE:
    net = ConvODEUNet(num_filters=16, output_dim=2, time_dependent=True, 
                      non_linearity='lrelu', adjoint=True, tol=1e-3)
    net.to(device)

In [ ]:
if TRAIN_RESNET:
    net = ConvResUNet(num_filters=16, output_dim=2, non_linearity='lrelu')
    net.to(device)

In [ ]:
if TRAIN_UNET:
    net = Unet(depth=BL, num_filters=64, output_dim=2).cuda()
    net.to(device)

## Modell zeigen

In [ ]:
from torchsummary import summary
#-# summary = summary(net, (3,512, 512), device='cuda')
summary = summary(net, SHAPE_IMG, device=devicename)

## Verarbeitung

---

In [ ]:
for m in net.modules():
    if isinstance(m, torch.nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.constant_(m.bias, 0)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(net)

# Train model

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
val_criterion = torch.nn.BCEWithLogitsLoss()

if TRAIN_UNET:
    cross_entropy = torch.nn.BCEWithLogitsLoss()

    def criterion(conf, labels):
        out_shape = conf.shape[2:4]
        label_shape = labels.shape[2:4]

        w = (label_shape[1] - out_shape[1]) // 2
        h = (label_shape[1] - out_shape[1]) // 2
        dh, dw = out_shape[0:2]

        conf_loss_ce = cross_entropy(conf, labels[:, :, h:h+dh, w:w+dw])

        return conf_loss_ce

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
losses = []
val_losses = []
nfe = [[],[],[],[],[],[],[],[],[]] if TRAIN_UNODE else None

In [ ]:
accumulate_batch = 8  # 8  # mini-batch size by gradient accumulation
accumulated = 0

if TRAIN_RESNET: filename = 'best_border_resnet_model.pt'
elif TRAIN_UNODE: filename = 'best_border_unode_model.pt'
elif TRAIN_UNET: filename = 'best_border_unet_model.pt'

def run(lr=1e-3, epochs=100):
    accumulated = 0
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    for epoch in range(epochs):
        
        # training loop with gradient accumulation
        running_loss = 0.0
        optimizer.zero_grad()
        for data in tqdm(trainloader):
            inputs, labels = data[0].cuda(), data[1].cuda()
            outputs = net(inputs)
            loss = criterion(outputs, labels) / accumulate_batch
            loss.backward()
            accumulated += 1
            if accumulated == accumulate_batch:
                optimizer.step()
                optimizer.zero_grad()
                accumulated = 0

            running_loss += loss.item() * accumulate_batch

        losses.append(running_loss / len(trainloader))
        
        # validation loop
        with torch.no_grad():
            running_loss = 0.0
            for data in valloader:
                inputs, labels = data[0].cuda(), data[1].cuda()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item()

            val_losses.append(running_loss / len(valloader))
            # Bedingung zum Speichern des Modells
            if np.argmin(val_losses) == len(val_losses) - 1 and loss < 0.4:
                torch.save(net, filename)
                #------Protokoll--------------------------
                protokolldatei = open('_protokoll.txt','a') #+
                protokolldatei.write('---------------------------------------------\n')  #+
                protokolldatei.write(f'Speicherung des Modells nach {epoch} Epochen, loss: {loss}\n') #+
                protokolldatei.close() #+
                #-------------------------------------
                
            clear_output(wait=True)
            plot_losses(inputs, outputs, losses, val_losses, get_title(), nfe, net=net)

In [ ]:
import datetime
protokoll_train = open(f'(4-1)_protokoll{SHAPE_IMG}.txt', mode='w')
protokoll_train.write(f'--- Projekt 3-1 --- Shape: {SHAPE_IMG}, {SHAPE_IMG}, Epochen: {EP}, Lernrate: {LR}  ---\n')

In [ ]:
if TRAIN_UNODE or TRAIN_RESNET: lr = LR * 10 # 1e-3
else: lr = LR # 1e-4
protokoll_train.write(f'Training start: {datetime.datetime.now()}\n')
# Training starten
run(lr, EP - len(losses))
protokoll_train.write(f'Training end:   {datetime.datetime.now()}\n')
protokoll_train.close()

## Calculate results

In [ ]:
# load best model
net = torch.load(filename)

In [ ]:
with torch.no_grad():
    running_loss = 0.0
    for data in tqdm(valloader):
        inputs, labels = data[0].cuda(), data[1].cuda()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

    print("Check validation loss:", running_loss / len(valloader))

## Visualize results on validation set

In [ ]:
#+ from inference_utils import inference_image, postprocess
from inference_utils_crag import inference_image, postprocess #+
import numpy as np              #+
import matplotlib.pyplot as plt #*

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=3, figsize=(4*3,3*5))

ax[0, 0].set_title('Image')
ax[0, 1].set_title('Ground-truth')
ax[0, 2].set_title(get_title())

for col in range(3):
    for row in range(5):
        index = val_set_idx[row]
        print(f'idx: {index}, ', end='')
        image = PIL.Image.open(path_images /  f'train_{index}.png')
        gt = PIL.Image.open(path_targets / f'train_{index}.png')
        
        with torch.no_grad():
            #? result, input_image = inference_image(net, image, shouldpad=TRAIN_UNET)
            result, input_image  = inference_image(net, image, shouldpad=TRAIN_UNET)
            result = postprocess(result, gt)
        if col == 0:
            ax[row, col].imshow(image)
        elif col == 1:
            ax[row, col].imshow(np.array(gt) > 0)
        else:
            ax[row, col].imshow(image)
            ax[row, col].imshow(result, alpha=0.5)
                
        ax[row, col].set_axis_off()
        


plt.show(); 

# Calculate metrics on test set

In [ ]:
from metrics import ObjectDice, ObjectHausdorff, F1score
import torch
import numpy as np
import PIL
import skimage.measure
from tqdm import tqdm, tqdm_notebook
from inference_utils_crag import inference_image, postprocess
import matplotlib.pyplot as plt
########################################################
from img_array_transform_jh import ArrayTransform as TRANSFORM
from _path import Path as PATH   # Pfade und Dateinamen
path=PATH() # Instanz der Klasse für Methodenaufruf erforderlich

In [ ]:
TEST_RESNET = False
TEST_UNODE = False
TEST_UNET = True

In [ ]:
if TEST_UNODE: net = torch.load('best_border_unode_model.pt')
elif TEST_RESNET: net = torch.load('best_border_resnet_model.pt')
elif TEST_UNET: net = torch.load('best_border_unet_model.pt')

In [ ]:
path_testimages=PATH.dataset / 'valid/Images/'
path_testtargets=PATH.dataset / 'valid/Annotation/'

filenames_testinputs  =path.get_filenames(path=path_testimages , dateifilter= '*.png')
filenames_testtargets =path.get_filenames(path=path_testtargets ,dateifilter='*.png')

print('Anzahl der Bilder      : ', len(filenames_testinputs))
print('Anzahl der Annotationen: ', len(filenames_testtargets))

anzahl_testimages=len(filenames_testinputs)

In [ ]:
# Visualisierung (jh)

idx=12
fname=f'test_{idx}.png'

image = PIL.Image.open(path_testimages /  fname)
gt = PIL.Image.open(path_testtargets / fname)

result, resized = inference_image(net, image, shouldpad=TEST_UNET)
result = postprocess(result, gt)
gt = skimage.measure.label(np.array(gt))

print('result: ', result.shape)
print('gt: ', gt.size)
print('image: ', image.size)



In [ ]:
# Visualisierung (jh)
transform=TRANSFORM()

plt.figure(figsize=(12,12))
plt.subplot(221).set_title('Image')
plt.imshow(image)

# Achtung: Drüsen der tatsächliche Maske sind indiziert !!
plt.subplot(222).set_title('tatsächliche Maske') 
plt.imshow(gt)

# Klassifizierung (0- Hintergrund, 1- Drüsen)
plt.subplot(223).set_title('tatsächliche Maske, binär') 
plt.imshow(transform.twoClasses(gt))

plt.subplot(224).set_title('prognostizierte Maske, binär')
plt.imshow(transform.twoClasses(result))

### Identnummernbezogene/Objektbezogene Bewertung
[ Id0 - Hintergrund, Durchnummerierung der Drüsen je maske von Id=1, 2 , 3 , ..., n

In [ ]:
protokolldatei = open(f'protokoll_prognose{SHAPE_MASK}.txt','w')

protokolldatei.write('\nObjektbezogenen Kennzahlen:\n')
# Tabellenkopf
print_string=(f' idx | dice_img  | f1_img    | hausdorff_img ')
protokolldatei.write(print_string+'\n')

In [ ]:

dice, hausdorff, f1, dice_full = 0, 0, 0, 0

if TEST_UNODE: folder = 'results_unode'
elif TEST_UNET: folder = 'results_unet'
elif TEST_RESNET: folder = 'results_resnet'
    
names = []
i_error=0
anzahl=0

for index in np.arange(1, anzahl_testimages+1):
    names.append(f'test_{index}.png')
  
 
for i, fname in tqdm_notebook(enumerate(names), total=anzahl_testimages):
    # tqdm.notebook.tqdm 
    # print(f'idx: {i}, ', end='') #
    image = PIL.Image.open(path_testimages /  fname)
    gt = PIL.Image.open(path_testtargets / fname)
    
    result, resized = inference_image(net, image, shouldpad=TEST_UNET)
    result = postprocess(result, gt)2.201
    
    
    gt = skimage.measure.label(np.array(gt))
    

    #-# f1_img, hausdorff_img, dice_img =0, 0, 0
    try:

        f1_img = F1score(result, gt)
        hausdorff_img = ObjectHausdorff(result, gt)
        dice_img = ObjectDice(result, gt)
        
        f1 += f1_img
        hausdorff += hausdorff_img
        dice += dice_img
        print(i,', ', fname,' : ', f1_img, hausdorff_img, dice_img)
        anzahl +=1
        
        #------Protokoll--------------------------
        print_string=(f' {i:3d} | {dice_img:9.3f} | {f1_img:9.3f} | {hausdorff_img:13.3f} ')
        print(print_string)
        # --- Protokoll ---
        protokolldatei.write(print_string+'\n')

    except:
        i_error +=1
        print('Error: ',i_error, 'Zyklus: ', i, 'Dateiname: ', fname)
    


In [ ]:
print('--Mittelwerte, Drüsen- bzw. Identnummernbezogen------------------------------------')
print('ObjectDice:', dice / anzahl )
print('Hausdorff:', hausdorff / anzahl)
print('F1:', f1 / anzahl )
print('Anzahl io.: ', anzahl)
print('Errors: ', i_error)

In [ ]:
protokolldatei.write('\n Mittelwerte, Objektbezogenen Kennzahlen (OB):\n')
protokolldatei.write(f'ObjectDice: { dice / anzahl }\n')
protokolldatei.write(f'Hausdorff: {hausdorff / anzahl}\n')
protokolldatei.write(f'F1: {f1 / anzahl}\n')

### Klassenbezogene Bewertung
[0 - Hintergrund, 1- Drüsen]

In [ ]:
protokolldatei.write('\nKlassenbezogenen Kennzahlen (KB):\n')
# Tabellenkopf
print_string=(f' idx | dice_img  | f1_img    | hausdorff_img ')
protokolldatei.write(print_string+'\n')

In [ ]:
dice, hausdorff, f1, dice_full = 0, 0, 0, 0

if TEST_UNODE: folder = 'results_unode'
elif TEST_UNET: folder = 'results_unet'
elif TEST_RESNET: folder = 'results_resnet'
    
names = []
i_error=0
anzahl=0

for index in np.arange(1, anzahl_testimages+1):
    names.append(f'test_{index}.png')
  
 
for i, fname in tqdm_notebook(enumerate(names), total=anzahl_testimages):
    # tqdm.notebook.tqdm 
    # print(f'idx: {i}, ', end='') #
    image = PIL.Image.open(path_testimages /  fname)
    gt = PIL.Image.open(path_testtargets / fname)
    
    result, resized = inference_image(net, image, shouldpad=TEST_UNET)
    result = postprocess(result, gt)
    
    
    gt = skimage.measure.label(np.array(gt))
    
    # To Binaer, da nur Hintergund (0) und Drüsen (1) (JH)
    gt= transform.twoClasses(gt)
    result= transform.twoClasses(result)

    #-# f1_img, hausdorff_img, dice_img =0, 0, 0
    try:

        f1_img = F1score(result, gt)
        hausdorff_img = ObjectHausdorff(result, gt)
        dice_img = ObjectDice(result, gt)
        
        f1 += f1_img
        hausdorff += hausdorff_img
        dice += dice_img
        print(i,', ', fname,' : ', f1_img, hausdorff_img, dice_img)
        anzahl +=1
        
        #------Protokoll--------------------------
        print_string=(f' {i:3d} | {dice_img:9.3f} | {f1_img:9.3f} | {hausdorff_img:13.3f} ')
        print(print_string)
        # --- Protokoll ---
        protokolldatei.write(print_string+'\n')
        #-------------------------------------

    except:
        i_error +=1
        print('Error: ',i_error, 'Zyklus: ', i, 'Dateiname: ', fname)
    
  

In [ ]:
print('--Mittelwerte, Klassenbezogen -----------------------------------')
print('ObjectDice:', dice / anzahl )
print('Hausdorff:', hausdorff / anzahl)
print('F1:', f1 / anzahl )
print('Anzahl io.: ', anzahl)
print('Errors: ', i_error)

In [ ]:
protokolldatei.write('\n Mittelwerte, klassenbezogene Kennzahlen (KB): \n')
protokolldatei.write(f'ObjectDice: {dice / anzahl}\n')
protokolldatei.write(f'Hausdorff: {hausdorff / anzahl}\n')
protokolldatei.write(f'F1: {f1 / anzahl}\n')

In [ ]:
protokolldatei.close()

## Visualisierung 1 mit Zwischendarstellungen

In [ ]:
def showPrognose(result_prog):
    '''Zum Visualisieren des Prognoseergbnisses ohne Nachbearbeitung (ohne Postprozess) (JH)'''
    #print(result_1.shape)
    kanal=np.zeros((1,result_prog.shape[1], result_prog.shape[2]))
    result_prog=np.concatenate((result_prog, kanal))
    result_prog=result_prog.transpose((2,1,0))
    result_prog=np.rot90(result_prog,k=1)
    result_prog=np.flip(result_prog, 0)
    return result_prog

In [ ]:
images, gts, results_prog, results=[],[],[],[]
for idx in range(len(filenames_testinputs)):
  #image = PIL.Image.open(filenames_testinputs[idx])
  #gt = PIL.Image.open(filenames_testtargets[idx])
  fnameimage= str(path_testimages) + f'/test_{idx+1}.png'
  fnamemask= str(path_testtargets) + f'/test_{idx+1}.png'
  image = PIL.Image.open(fnameimage)
  gt = PIL.Image.open(fnamemask) 
  with torch.no_grad():
      # Prognose
      result_prog, input_image = inference_image(net, image, shouldpad=TRAIN_UNET)
      # Nachbearbeitung
      result = postprocess(result_prog, gt)
  images.append(image)
  gts.append(gt)

  # Zum Visualisieren des Prognoseergbnisses ohne Nachbearbeitung 
  result_prog=showPrognose(result_prog)

  results_prog.append(result_prog)
  results.append(result)
 

In [ ]:
from importlib import reload
import MODULE.JH.visualize
reload(MODULE.JH.visualize)

from MODULE.JH.visualize import Show as SHOW

show=SHOW(experiment='', figsize=(15,15), fontsize=16)

In [ ]:
idx_list=list([12,22,26])
path_set= f'(4-1)_images_masks_listset_{str(idx_list)}.png'

listset=list([images, gts, results_prog, results])
titles=list(['Image', 'tatsächliche Maske','prognostizierte Maske', 'mit Postprocessing'])

show.list_set(idx_list, listset,titles, path=path_set)

### Alternative Darstellung

In [ ]:
show=SHOW(experiment='', figsize=(15,11), fontsize=14)
path_set= f'(1-1)_images_masks_Objekte_{str(idx_list)}.png'

listset=list([images, gts, results])
titles=list(['Image', 'tatsächliche Maske','Prognose mit Postprocessing'])

show.list_set(idx_list, listset,titles, path=path_set)

### Visualisierung 2, horizontal

In [ ]:
# Ergänzung: Ausgabe von Images und Masken für das Skript (JH)

path_testimages=PATH.dataset / 'valid/Images/'
path_testtargets=PATH.dataset / 'valid/Annotation/'

fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(4*3,4*3))
samples=[6, 26, 23]


for col in range(3):
    for row in range(3):
        #index = val_set_idx[samples[row]]
        ax[row, 0].set_title(f'Image {samples[row]}')
        ax[row, 1].set_title(f'tatsächliche Maske {samples[row]}')
        ax[row, 2].set_title(f'prognostizierte Maske {samples[row]}')
        
        index = samples[row]
        
        #fname=f'test_{idx}.png'
        fnameimage= str(path_testimages) + f'/test_{index+1}.png'
        fnamemask= str(path_testtargets) + f'/test_{index+1}.png'
        
      
        image = PIL.Image.open(fnameimage)
        gt = PIL.Image.open(fnamemask)
        
        with torch.no_grad():
            result, input_image = inference_image(net, image, shouldpad=TRAIN_UNET)
            result = postprocess(result, gt)
        
        
        with torch.no_grad():
            result, input_image = inference_image(net, image, shouldpad=TRAIN_UNET)
            result = postprocess(result, gt)
        if col == 0:
            ax[row, col].imshow(image)
        elif col == 1:
            #ax[row, col].imshow(np.array(gt) > 0)
            ax[row, col].imshow(np.array(gt))
        else:
            ax[row, col].imshow(image)
            ax[row, col].imshow(result, alpha=1)
                
        ax[row, col].set_axis_off()

plt.savefig(f'Test_Image_Maske_Prognose{samples}.png', bbox_inches="tight")
plt.show(); 